# MSR GeoLife Data
The first part contains code to get your hands on the data.
This dataset can get really big, so the second part will contain a function to help with gathering data for exploration.

The third part will then contain code examples of what you can do with the data.

This notebook has a baseline model to classify trajectories into different vehicles.

# Part I. Getting Data from Blob Storage

In [17]:
from azure.storage.blob import BlockBlobService
import os
import numpy as np
import pandas as pd
from datetime import datetime
import sklearn

In [4]:
!mkdir geolife

mkdir: cannot create directory ‘geolife’: File exists


In [5]:
local_path=os.getcwd() + "/geolife"
blob_account_name = "mldsdatahack2019diag" # fill in your blob account name
blob_account_key = "JsauBssnY92CeD3MgI2SWhkQ16JioJCRWVW8NzKtcWckI+DaNNbCmpmMAVq27GD91mhgH+oHPx+QbIKUCow5gA=="  # fill in your blob account key
mycontainer = "datahackdata2019"       # fill in the container name 
myblobname = "000/Trajectory/20090705025307.csv"        # fill in the blob name 
mydatafile = "Output"        # fill in the output file name

In [6]:
import azure
from azure.storage.blob import BlockBlobService

blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)
containers = blob_service.list_containers()
blobs = [a for a in blob_service.list_blobs("datahackdata2019")]
csv_names = [a.name for a in blobs]

In [7]:
for name in csv_names:
    if name == "BCycleAustin.csv": continue
    blob_service.get_blob_to_path("datahackdata2019", name, os.path.join(local_path, name))

## Segregate users with and without transportation data

In [28]:
data_dir = './geolife'
users = sorted([x.split(".")[0] for x in os.listdir(data_dir) if x.endswith(".csv")])
print(users) # from 000 to 181

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142'

In [ ]:
# Some users do not have transport informatio
users_without_transport, users_with_transport = [],[]
for u in users:
    df_usr =  pd.read_csv(data_dir + '/' + u + '.csv')
    if len(df_usr['Transportation Mode'].unique()) == 1:
        if type(df_usr['Transportation Mode'].unique()[0]) == np.float64:
            users_without_transport.append(u)
            continue
    users_with_transport.append(u)

/anaconda/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Get the relevant features for transport mode classification
The basic features used in this baseline are:
- dist_x: distance travelled in longitude
- dist_y: distance travelled in llatitude
- time_diff: time travelled 
- avg_vel_x: average velocity in x direction(longitude)
- avg_vel_y: average velocity in y direction(latitude)

In [14]:
"""Data preprocessing"""

def get_cont_lab_indices(df_usr1):
    """
    Returns indices at which a particular transport mode starts
    """
    cont_label_indices = []
    for idx in range(len(df_usr1)):
        if idx == 0 or (df_usr1['Transportation Mode'][idx] != df_usr1['Transportation Mode'][idx-1]):
            entry = []
            entry.append(idx)
            entry.append(df_usr1['Transportation Mode'][idx])
            cont_label_indices.append(entry)
    return cont_label_indices

def get_dists_and_velocities(indices, df_usr1):
    """
    Function for calculating relevant velocity and distance features
    """
    start,end = indices[0][0],indices[1][0]
    dist_x = abs(df_usr1['Longitude'][end-1] - df_usr1['Longitude'][start])
    dist_y = abs((df_usr1['Latitude'][end-1] - df_usr1['Latitude'][start]))
    dt = pd.to_datetime(df_usr1['DateTime'])
    time_diff = (datetime.fromtimestamp(dt[end-1].timestamp()) - datetime.fromtimestamp(dt[start].timestamp())).total_seconds()
    avg_vel_x = dist_x/time_diff * 10000000 #arbitrary constant 
    avg_vel_y = dist_y/time_diff  * 10000000
    return [dist_x,dist_y,time_diff, avg_vel_x, avg_vel_y]
    
def get_features(user):
    """
    Extact features for a particular user
    """
    directory = "data_challenge/"
    if not os.path.exists(directory):
        os.mkdir(directory)
    df_usr =  pd.read_csv(data_dir + '/' + user + '.csv')
    df_usr1 = df_usr.dropna(subset = ['Transportation Mode']).reset_index()
    lst = get_cont_lab_indices(df_usr1)
    header = ['dx','dy','delta_t','vx','vy','label']
    df = pd.DataFrame(columns = header)
    df_idx = 0
    for i in range(len(lst) -1):
        if lst[i+1][0] - lst[i][0] <=1:
            continue
        feats = get_dists_and_velocities([lst[i],lst[i+1]], df_usr1)
        lab = lst[i][1]
        #ignore the entries with more than one label
        if len(lab.split(",")) >=2:
            continue
        feats.append(lab)
        df.loc[df_idx] = feats
        df_idx += 1
    df.to_csv(directory + user +".csv")
total = len(users_with_transport)
for idx,user in enumerate(users_with_transport):
    print(str(idx+1) +"/"+ str(total))
    get_features(user)

1/64


/anaconda/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


2/64
3/64
4/64
5/64
6/64
7/64
8/64
9/64
10/64
11/64


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:25: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:26: RuntimeWarning: divide by zero encountered in double_scalars


12/64
13/64
14/64
15/64
16/64
17/64
18/64
19/64
20/64
21/64
22/64
23/64
24/64
25/64
26/64
27/64
28/64
29/64
30/64
31/64
32/64
33/64
34/64
35/64
36/64
37/64
38/64
39/64
40/64
41/64
42/64
43/64
44/64
45/64
46/64
47/64
48/64
49/64
50/64
51/64
52/64
53/64


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in double_scalars


54/64
55/64
56/64
57/64
58/64
59/64
60/64
61/64
62/64
63/64
64/64


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:26: RuntimeWarning: invalid value encountered in double_scalars


In [15]:
def merge_dfs(users):
    """
    Merge user data frames into a single one
    """
    directory = "data_challenge/"
    combined_df = pd.DataFrame()
    for user in users_with_transport:
        df_usr =  pd.read_csv(directory + user + '.csv' , header = None)
        combined_df = pd.concat([combined_df, df_usr[1:]])
    combined_df.to_csv(directory + "combined.csv", index= False)
    return combined_df
cdf = merge_dfs(users_with_transport).reset_index()
       

In [16]:
"""Random forest classifier"""
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
le = LabelEncoder()
Y = cdf.iloc[:,7]
le.fit(Y)
print(list(le.classes_))
Y  = le.transform(Y)  #label encoding
X = cdf.iloc[:,2:7]
X   = X.as_matrix().astype(float)
nans = []

#removal of NaN and Infs 
for idx,i in enumerate(X):
    for ikx,j in enumerate(i):
        if not np.isfinite(j):
            nans.append(idx)
nans = np.unique(np.asarray(nans))
X = np.delete(X, nans, axis = 0)
Y =  np.delete(Y, nans)   
assert len(X) == len(Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0)
clf=RandomForestClassifier(n_estimators=500)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("Accuracy:")
print((y_pred == y_test).mean() * 100)
a,b = np.unique(y_test, return_counts=True)
print("Trivial Accuracy:")
print(np.max(b)/np.sum(b) * 100)

['airplane', 'bike', 'boat', 'bus', 'car', 'motorcycle', 'run', 'subway', 'taxi', 'train', 'walk']


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Accuracy:
71.4359238291302
Trivial Accuracy:
41.945445187853835
